# Transmission line simulation with pysonnet
Let's do some imports

In [1]:
import pysonnet as ps
import numpy as np
# gdspy is availible via 'pip install gdspy'
# or at https://github.com/heitzmann/gdspy
import gdspy as gp

## Make a simple geometry with gdspy
We can make a cool wavy transmission line using the gdspy.Path class

The result can be easily viewed using gdspy's built in layout viewer (uncomment the last line)

In [2]:
# geometric constants
box_x, box_y = 100, 100  # size of the box
width = 5  # transmission line width
layer = 0  # layer for gds format
datatype = 1  # datatype for gds format
# initialize the path
path = gp.Path(width, initial_point=(box_x / 3, 0))
# define the cosine path and it's derivative
def path_function(t):
    return (-box_y / 6 * (np.cos(2 * np.pi * t) - 1), t * box_y)
def d_path_function(t):
    return (np.pi * box_y / 3 * np.sin(2 * np.pi * t), box_y)
path.parametric(path_function, curve_derivative=d_path_function,
                layer=layer, datatype=datatype)
# add the path to a cell
cell = gp.Cell('feedline', exclude_from_current=True)
cell.add(path);
# show the result
# gp.LayoutViewer(cells=cell)

## Configure the project settings
The first step is to pick all the settings that we want for the sonnet simulation

In [3]:
project = ps.GeometryProject()  # this is the most basic sonnet project type
# I don't like the default Sonnet units, so let's change them
project.set_units(length='um')  # the default length unit for Sonnet is mils
# Then we can set up the box
project.setup_box(box_x, box_y, 50, 50)
project.set_origin(0, -box_y)  # Sonnet likes the origin at the top left
# Next is defining the metal types and if they are on the box top/bottom
project.define_metal('free space', top=True)  # Sonnet considers 'free space' a metal
project.define_metal("lossless", bottom=True)
project.define_metal("general", name="PtSi", ls=21)  # high kinetic inductance superconductor
# Lets add a dielectric layer under the microstrip and air above
project.add_dielectric("air", 0)
project.add_dielectric("silicon", 1, epsilon=11.9, dielectric_loss=0.004, conductivity=4.4e-4)

## Load the geometry into a pysonnet project
The polygons can be extracted using the get_polygons() method for the Cell, CellReference or CellArray.

by_spec needs to be True for the format to be compatible with pysonnet

In [4]:
polygons = cell.get_polygons(by_spec=True)
print(polygons[(layer, datatype)])

[array([[ 30.83333333,   0.        ],
       [ 30.8731849 ,   1.18776734],
       [ 30.99228514,   2.37263598],
       [ 31.18931432,   3.55194109],
       [ 31.46220701,   4.72344248],
       [ 31.80831886,   5.88544282],
       [ 32.22460491,   7.0368249 ],
       [ 32.70777873,   8.17701841],
       [ 33.25443368,   9.3059187 ],
       [ 33.86112078,  10.42378229],
       [ 34.52438741,  11.53111978],
       [ 35.24078658,  12.62859989],
       [ 36.00686731,  13.71697124],
       [ 36.81915544,  14.79700352],
       [ 37.67413176,  15.86944619],
       [ 38.56821188,  16.93500185],
       [ 39.49773018,  17.9943105 ],
       [ 40.45892893,  19.04794177],
       [ 41.44795264,  20.09639236],
       [ 42.4608472 ,  21.14008683],
       [ 43.49356329,  22.17938015],
       [ 44.54196332,  23.21456107],
       [ 45.60183129,  24.24585562],
       [ 46.66888501,  25.27343028],
       [ 47.73879019,  26.29739454],
       [ 48.80717611,  27.31780271],
       [ 49.86965254,  28.33465477],


The layer and datatype kewords allow us to assign groups of polygons to different sonnet layers with different polygon properties. 